In [2]:
import pandas as pd

import requests
import json
from bs4 import BeautifulSoup
import time

from tqdm import tqdm

# 좌표 -> 행정동

- region_type : H(행정동) 또는 B(법정동)

---

* 지하철, 버스정류장, 경찰청, 단독.다가구

In [2]:
# 지하철 역
subway = pd.read_csv("data/지하철변환/지하철주소.csv", encoding="cp949")
subway.head()

,No,연번,역번호,호선,호선.1,역명,역전화번호,역주소,경도,위도,상태
0,1,1,150,1,01호선,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가),126.972554,37.557160,정좌표
1,2,2,151,1,01호선,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동),126.976983,37.565438,정좌표
2,3,3,152,1,01호선,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가),126.983240,37.570214,정좌표
3,4,4,153,1,01호선,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가),126.992029,37.570426,정좌표
4,5,5,154,1,01호선,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가),127.001916,37.570908,정좌표


In [3]:
# 불필요한 컬럼 제외
subway.drop(["No", "연번", "호선.1", "상태"], axis = 1, inplace=True)
subway.head()

,역번호,호선,역명,역전화번호,역주소,경도,위도
0,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가),126.972554,37.557160
1,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동),126.976983,37.565438
2,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가),126.983240,37.570214
3,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가),126.992029,37.570426
4,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가),127.001916,37.570908


In [4]:
# 구, 행정동 컬럼 추가
subway['구'] = 0
subway['행정동'] = 0
subway.head()

,역번호,호선,역명,역전화번호,역주소,경도,위도,구,행정동
0,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가),126.972554,37.557160,0,0
1,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동),126.976983,37.565438,0,0
2,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가),126.983240,37.570214,0,0
3,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가),126.992029,37.570426,0,0
4,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가),127.001916,37.570908,0,0


In [5]:
url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
key = '자신의 key 넣을 것'
header = {'Authorization': 'KakaoAK {}'.format(key)}

            # 경도, 위도 / X좌표, Y좌표, 상황에 맞춰 수정할 것
for x, y in subway[["경도", "위도"]].values:
    queryString = {'x' : str(x), 'y' : str(y)}
    r = requests.get(url, headers=header, params=queryString)
    time.sleep(0.5)
    data = json.loads(r.text)
    
    hangjung = data["documents"][1]
    
                    # 경도, 위도 / X좌표, Y좌표, 상황에 맞춰 수정할 것
    subway.loc[(subway["경도"] == x) & (subway["위도"] == y), "구"] = hangjung["region_2depth_name"]
    subway.loc[(subway["경도"] == x) & (subway["위도"] == y), "행정동"] = hangjung["region_3depth_name"]

In [6]:
subway

,역번호,호선,역명,역전화번호,역주소,경도,위도,구,행정동
0,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가),126.972554,37.557160,중구,회현동
1,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동),126.976983,37.565438,중구,소공동
2,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가),126.983240,37.570214,종로구,종로1.2.3.4가동
3,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가),126.992029,37.570426,종로구,종로1.2.3.4가동
4,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가),127.001916,37.570908,종로구,종로5.6가동
...,...,...,...,...,...,...,...,...,...
287,4134,9,송파나루,02-2656-0934,서울특별시 송파구 오금로 165,127.113123,37.510120,송파구,방이2동
288,4135,9,한성백제,02-2656-0935,서울특별시 송파구 위례성대로 51,127.118185,37.516865,송파구,오륜동
289,4136,9,올림픽공원,02-2656-0936,서울특별시 송파구 양재대로 1233,127.130600,37.516088,송파구,오륜동
290,4137,9,둔촌오륜,02-2656-0937,서울특별시 송파구 강동대로 327,127.140170,37.517975,송파구,오륜동


In [8]:
subway.to_csv("data/행정동변환/지하철주소_행정동변환(cp949).csv", encoding="cp949", index = False)

In [9]:
pd.read_csv("data/행정동변환/지하철주소_행정동변환(cp949).csv", encoding="cp949")

,역번호,호선,역명,역전화번호,역주소,경도,위도,구,행정동
0,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가),126.972554,37.557160,중구,회현동
1,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동),126.976983,37.565438,중구,소공동
2,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가),126.983240,37.570214,종로구,종로1.2.3.4가동
3,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가),126.992030,37.570426,종로구,종로1.2.3.4가동
4,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가),127.001916,37.570908,종로구,종로5.6가동
...,...,...,...,...,...,...,...,...,...
287,4134,9,송파나루,02-2656-0934,서울특별시 송파구 오금로 165,127.113123,37.510120,송파구,방이2동
288,4135,9,한성백제,02-2656-0935,서울특별시 송파구 위례성대로 51,127.118185,37.516865,송파구,오륜동
289,4136,9,올림픽공원,02-2656-0936,서울특별시 송파구 양재대로 1233,127.130600,37.516088,송파구,오륜동
290,4137,9,둔촌오륜,02-2656-0937,서울특별시 송파구 강동대로 327,127.140170,37.517975,송파구,오륜동


# 주소 ->  행정동

* 매매, 전월세 데이터 실행 할 것(단독, 다가구 제외)
* 표준공시지가

---

* 중간에 중지되어 다시 실행하는 경우<br>
    -> 파일 이름 data/부동산/연립다세대_매매_{}(cp949).csv 로 수정할 것<br>
    -> 도로명주소 column만들기부터 print("----",year, "년도----") 직전까지 주석 처리<br>
    -> [attachedHouseSales["행정동"] == 0]의 0을 모두 str 처리 할 것 '0'(22, 25, 40, 54번째 줄)<br><br><br>
    
* 표준공시지가의 경우<br>
    -> for year~~ 다음의 셀을 사용할 것

In [14]:
url = 'https://dapi.kakao.com/v2/local/search/address.json'
key = '자신의 key 넣을 것'
header = {'Authorization': 'KakaoAK {}'.format(key)}

In [4]:
for year in range(2011, 2021):
    # 파일 불러오기
    attachedHouseSales = pd.read_csv("data/부동산/연립다세대매매/연립다세대(매매)__실거래가_{}.csv".format(year), encoding = "cp949", header = 15)
    
    # 도로명주소 column만들기
    attachedHouseSales['도로명주소'] = '서울특별시 ' + attachedHouseSales["시군구"].str.split().str[1] + " " + attachedHouseSales["도로명"]
    
    # 지번 주소 column 생성
    attachedHouseSales["지번주소"] = attachedHouseSales["시군구"] + attachedHouseSales["번지"]
    
    # 필요없는 column 삭제
    attachedHouseSales.drop(["번지", "본번", "부번", "도로명"], axis = 1, inplace = True)
    
    #구, 동 컬럼 생성
    # 구에는 시군구에서 구 잘라 붙이기
    attachedHouseSales["구"] = attachedHouseSales["시군구"].str.split().str[1]
    ## 행정동은 채워넣을 공간 만들기
    attachedHouseSales['행정동'] = 0
    
    print("----------------------", year, "년도-------------------------")
    print("총 row 개수 : ", attachedHouseSales.shape[0])
    print("바꿔야할 주소의 개수 : ",  len(attachedHouseSales["지번주소"][attachedHouseSales["행정동"] == '0'].unique()))
    
    ## 지번주소로 한번 바꿔주기
    for i in tqdm(attachedHouseSales[attachedHouseSales["행정동"] == '0']["지번주소"].unique()):
        queryString = {'query' : i}
        r = requests.get(url, headers=header, params=queryString)
        time.sleep(0.2)
        data = json.loads(r.text)

        if len(data["documents"]) == 0:
            continue

        if data["documents"][0]["address"] == None:
            continue

        attachedHouseSales.loc[attachedHouseSales['지번주소'] == i, "행정동"] = data["documents"][0]["address"]['region_3depth_h_name']

    ## 지번주소로 안된 친구들로 혹시 모르니까 도로명 주소로도 바꿔주기
    for i in tqdm(attachedHouseSales[attachedHouseSales["행정동"] == '0']["도로명주소"].unique()):
        queryString = {'query' : i}
        r = requests.get(url, headers=header, params=queryString)
        time.sleep(0.1)
        data = json.loads(r.text)

        if len(data["documents"]) == 0:
            continue

        if data["documents"][0]["address"] == None:
            continue

        attachedHouseSales.loc[attachedHouseSales['도로명주소'] == i, "행정동"] = data["documents"][0]["address"]['region_3depth_h_name']

    print("변환안된 주소의 개수 : ", len(attachedHouseSales[attachedHouseSales["행정동"] == '0']["지번주소"].unique()))
    
    # 저장하기
    attachedHouseSales.to_csv("data/부동산/연립다세대_매매_{}(cp949).csv".format(year), encoding="cp949", index = False)
    print("☆ 저장 완료 ☆")

  0%|                                                                                         | 0/4352 [00:00<?, ?it/s]

---------------------- 2011 년도-------------------------
총 row 개수 :  31174
바꿔야할 주소의 개수 :  4352


100%|████████████████████████████████████████████████████████████████████████████████| 295/295 [00:53<00:00,  5.49it/s]


변환안된 주소의 개수 :  254
☆ 저장 완료 ☆


### 표준공시지가

In [9]:
publishPrice = pd.read_csv("data/부동산/표준공시지가_전처리1(cp949).csv", encoding="cp949")
publishPrice.head()

,시군구명,지목,면적,용도지역1,용도지역2,이용상황,주위환경,지세명,공시지가,주소
0,종로구,대,416.2,제1종일반주거지역,NaN,단독주택,기존 주택지대,완경사지,2300000,서울특별시 종로구 청운동 4-15
1,종로구,대,811.3,제1종일반주거지역,NaN,연립주택,일반 주택지대,급경사지,2150000,서울특별시 종로구 청운동 8-9
2,종로구,대,552.1,제1종일반주거지역,NaN,단독주택,고급 주택지대,평지,3280000,서울특별시 종로구 청운동 15-12
3,종로구,대,538.8,제1종일반주거지역,NaN,단독주택,고급 주택지대,완경사지,2880000,서울특별시 종로구 청운동 15-13
4,종로구,대,251.2,제1종일반주거지역,NaN,단독주택,일반 주택지대,완경사지,2100000,서울특별시 종로구 청운동 15-32


In [22]:
## 행정동 column 추가
publishPrice["행정동"] = 0
publishPrice.head()

,시군구명,지목,면적,용도지역1,용도지역2,이용상황,주위환경,지세명,공시지가,주소,행정동
0,종로구,대,416.2,제1종일반주거지역,NaN,단독주택,기존 주택지대,완경사지,2300000,서울특별시 종로구 청운동 4-15,0
1,종로구,대,811.3,제1종일반주거지역,NaN,연립주택,일반 주택지대,급경사지,2150000,서울특별시 종로구 청운동 8-9,0
2,종로구,대,552.1,제1종일반주거지역,NaN,단독주택,고급 주택지대,평지,3280000,서울특별시 종로구 청운동 15-12,0
3,종로구,대,538.8,제1종일반주거지역,NaN,단독주택,고급 주택지대,완경사지,2880000,서울특별시 종로구 청운동 15-13,0
4,종로구,대,251.2,제1종일반주거지역,NaN,단독주택,일반 주택지대,완경사지,2100000,서울특별시 종로구 청운동 15-32,0


In [26]:
# 저장안된 row의 개수
no_hang = 0

for i in tqdm(publishPrice["주소"]):
    queryString = {'query' : i}
    r = requests.get(url, headers=header, params=queryString)
    time.sleep(0.2)
    data = json.loads(r.text)

    if len(data["documents"]) == 0:
        no_hang += 1
        continue

    if data["documents"][0]["address"] == None:
        no_hang += 1
        continue

    publishPrice.loc[publishPrice['주소'] == i, "행정동"] = data["documents"][0]["address"]['region_3depth_h_name']

print("변환되지 않은 row의 개수 : ", no_hang)

publishPrice.to_csv("data/부동산/표준공시지가_전처리2(cp949).csv", encoding="cp949", index=False)
print("☆ 저장 완료 ☆")

100%|██████████████████████████████████████████████████████████████████████████| 24681/24681 [1:54:24<00:00,  3.60it/s]


변환되지 않은 row의 개수 :  923
☆ 저장 완료 ☆


# 도로명 주소 -> 좌표

* 단독,다가구 주택 실행 할 것, 실행한 후 위에 있는 좌표-> 행정동 코드 까지 실행 할 것

In [ ]:
url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
key = '자신의 key 넣을 것'
header = {'Authorization': 'KakaoAK {}'.format(key)}

In [ ]:
for year in range(2011, 2021):
    # 파일 불러오기
    jeonwolse_house = pd.read_csv("data/단독다가구/단독다가구(전월세)_실거래가_{}.csv".format(year), encoding = "cp949", header=16)
    
    # 주소값 없는 row 삭제
    jeonwolse_house = jeonwolse_house.dropna()

    # 도로명주소 column만들기
    jeonwolse_house['도로명주소'] = '서울특별시 ' + jeonwolse_house["시군구"].str.split().str[1] + " " + jeonwolse_house["도로명"]
    
    # 필요없는 column 삭제
    jeonwolse_house.drop(["번지", "도로명"], axis = 1, inplace = True)
    
    #구, 동 컬럼 생성
    ## 구에는 시군구에서 구 잘라 붙이기
    jeonwolse_house["구"] = jeonwolse_house["시군구"].str.split().str[1]
    ## 행정동은 채워넣을 공간 만들기
    jeonwolse_house['행정동'] = 0
    
    # x, y 좌표 채워넣을 공간 만들기
    jeonwolse_house['x'] = 0
    jeonwolse_house['y'] = 0
    
    print("----------------------", year, "년도-------------------------")
    print("총 row 개수 : ", jeonwolse_house.shape[0])
    print("바꿔야할 주소의 개수 : ",  len(jeonwolse_house["도로명주소"][jeonwolse_house["행정동"] == '0'].unique()))
    
    for i in tqdm(jeonwolse_house[jeonwolse_house["행정동"] == '0']["도로명주소"].unique()):
        queryString = {'query' : i}
        r = requests.get(url, headers=header, params=queryString)
        time.sleep(0.1)
        data = json.loads(r.text)

        if len(data["documents"]) == 0:
            continue

        if data["documents"][0]["road_address"] == None:
            continue

        jeonwolse_house.loc[jeonwolse_house['도로명주소'] == i, "x"] = data["documents"][0]["x"]
        jeonwolse_house.loc[jeonwolse_house['도로명주소'] == i, "y"] = data["documents"][0]["y"]

    print("변환안된 주소의 개수 : ", len(jeonwolse_house[jeonwolse_house["행정동"] == '0']["도로명주소"].unique()))
    
    # 저장하기
    jeonwolse_house.to_csv("data/부동산/단독다가구_전월세_좌표변환_{}(cp949).csv".format(year), encoding="cp949", index = False)
    print("☆ 저장 완료 ☆")
    
## 이후 저장한 데이터는 위에서 좌표 -> 행정동 부분 다시 돌릴 것